In [2]:
import tensorflow as tf
print(tf.__version__)

2.0.0


In [3]:
tf.test.is_gpu_available()

True

In [4]:
import tensorflow as tf
class MyDense(tf.keras.layers.Layer):
    def __init__(self, units=32, **kwargs):
        super(MyDense, self).__init__(**kwargs)
        self.units = units

    #build方法一般定义Layer需要被训练的参数。    
    def build(self, input_shape): 
        self.w = self.add_weight(shape=(input_shape[-1], self.units),
                                 initializer='random_normal',
                                 trainable=True,
                                 name='w')
        self.b = self.add_weight(shape=(self.units,),
                                 initializer='random_normal',
                                 trainable=True,
                                 name='b')
        super(MyDense,self).build(input_shape) # 相当于设置self.built = True

    #call方法一般定义正向传播运算逻辑，__call__方法调用了它。    
    def call(self, inputs): 
        return tf.matmul(inputs, self.w) + self.b

    #如果要让自定义的Layer通过Functional API 组合成模型时可以序列化，需要自定义get_config方法。
    def get_config(self):  
        config = super(MyDense, self).get_config()
        config.update({'units': self.units})
        return config

In [5]:

inputs = tf.keras.Input(shape=(4,))  
x = MyDense(units=16)(inputs) #tf.keras.layers.Dense()
x = tf.nn.tanh(x) 
x = MyDense(units=3)(x)
predictions = tf.nn.softmax(x)
model = tf.keras.Model(inputs=inputs, outputs=predictions)


In [6]:
from sklearn import datasets
iris = datasets.load_iris()
data = iris.data
labels = iris.target


In [7]:
import numpy as np

In [8]:
data = np.concatenate((data,labels.reshape(150,1)),axis=-1)
np.random.shuffle(data)

In [9]:
labels = data[:,-1]
data = data[:,:4]

In [10]:


model.compile(optimizer=tf.keras.optimizers.Adam(),
              loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True),
              metrics=[tf.keras.metrics.SparseCategoricalAccuracy()])


model.fit(data, labels, batch_size=32, epochs=10,shuffle=True)

Train on 150 samples
Epoch 1/10
150/150 [==============================] - 1s 7ms/sample - loss: 1.0987 - sparse_categorical_accuracy: 0.3333
Epoch 2/10
150/150 [==============================] - 0s 200us/sample - loss: 1.0968 - sparse_categorical_accuracy: 0.3067
Epoch 3/10
150/150 [==============================] - 0s 133us/sample - loss: 1.0946 - sparse_categorical_accuracy: 0.0733
Epoch 4/10
150/150 [==============================] - 0s 133us/sample - loss: 1.0928 - sparse_categorical_accuracy: 0.0467
Epoch 5/10
150/150 [==============================] - 0s 133us/sample - loss: 1.0910 - sparse_categorical_accuracy: 0.2800
Epoch 6/10
150/150 [==============================] - 0s 200us/sample - loss: 1.0889 - sparse_categorical_accuracy: 0.3333
Epoch 7/10
150/150 [==============================] - 0s 133us/sample - loss: 1.0864 - sparse_categorical_accuracy: 0.3333
Epoch 8/10
150/150 [==============================] - 0s 133us/sample - loss: 1.0839 - sparse_categorical_accuracy: 0.33

In [12]:
model.summary()

Model: "model"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         [(None, 4)]               0         
_________________________________________________________________
my_dense (MyDense)           (None, 16)                80        
_________________________________________________________________
tf_op_layer_Tanh (TensorFlow [(None, 16)]              0         
_________________________________________________________________
my_dense_1 (MyDense)         (None, 3)                 51        
_________________________________________________________________
tf_op_layer_Softmax (TensorF [(None, 3)]               0         
Total params: 131
Trainable params: 131
Non-trainable params: 0
_________________________________________________________________


In [13]:
model.save('keras_model_tf_version.h5')